<a href="https://colab.research.google.com/github/tihunn/emotion_testing/blob/main/prediction_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import numpy as np
import librosa
from tensorflow.keras.models import load_model

emotional_testing_model = load_model('emotional_testing.keras')
CLASS_LIST = ['neutral', 'disgust', 'sad', 'happy', 'surprise', 'angry', 'fear']
class_list_ru = ['нейтральный', 'отвращение', 'грустный', 'счастливый', 'удивление', 'гнев', 'страх']
N_FFT = 8192                              # Размер окна преобразования Фурье для расчета спектра
HOP_LENGTH = 512                          # Объем данных для расчета одного набора признаков


# Функция параметризации аудио
def get_features(y,                     # волновое представление сигнала
                 sr,                    # частота дискретизации сигнала y
                 n_fft=N_FFT,           # размер скользящего окна БПФ
                 hop_length=HOP_LENGTH  # шаг скользящего окна БПФ
                 ):
    chroma_stft = librosa.feature.chroma_stft(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)     # Хромаграмма
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)                   # Мел-кепстральные коэффициенты
    rmse = librosa.feature.rms(y=y, hop_length=hop_length)                                        # Среднеквадратическая амплитуда
    spec_cent = librosa.feature.spectral_centroid(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length) # Спектральный центроид
    spec_bw = librosa.feature.spectral_bandwidth(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)  # Ширина полосы частот
    rolloff = librosa.feature.spectral_rolloff(y=y, sr=sr, n_fft=n_fft, hop_length=hop_length)    # Спектральный спад частоты
    zcr = librosa.feature.zero_crossing_rate(y, hop_length=hop_length)                            # Пересечения нуля

    # Сборка признаков в общий список:
    features = {'rmse': rmse,
                'spct': spec_cent,
                'spbw': spec_bw,
                'roff': rolloff,
                'zcr' : zcr,
                'mfcc': mfcc,
                'stft': chroma_stft}

    return features

import numpy as np
import librosa

def prepare_prediction_batch(audio_file_path, duration_sec=2):
    y, sr = librosa.load(audio_file_path, mono=True)
    total_duration = librosa.get_duration(y=y, sr=sr)

    # Подготовка списка для хранения каждого 2-секундного отрезка
    segments = []

    # Разбиваем аудио на отрезки по `duration_sec` секунд
    for start in range(0, int(total_duration), duration_sec):
        end = start + duration_sec
        segment = y[start * sr : end * sr]

        # Если последний отрезок короче 1 секунды и не единственный, пропускаем его
        if len(segment) < sr and start > 0:
            continue

        # Если единственный отрезок короче 2 секунд, дополняем его до нужной длины
        if len(segment) < duration_sec * sr:
            repeats = int(np.ceil((duration_sec * sr) / len(segment)))
            segment = np.tile(segment, repeats)[:duration_sec * sr]

        # Получаем признаки для сегмента и добавляем в batch
        features = get_features(segment, sr)
        feature_vector = np.hstack([f.mean(axis=1) for f in features.values()])  # Конкатенируем признаки
        segments.append(feature_vector)

    # Возвращаем пакет признаков для всех сегментов в формате numpy массива
    return np.array(segments)

def format_time(seconds):
    """Форматирование времени в минуты и секунды."""
    if seconds < 60:
        return f"{seconds} сек"
    minutes = seconds // 60
    seconds = seconds % 60
    return f"{minutes} мин {seconds} сек"

def predict_emotion(prediction_batch, model, class_list, return_list=False):
    # Предсказания модели по каждому отрезку
    predictions = model.predict(prediction_batch)

    if return_list:
        prediction_with_times = []
        for i, prediction in enumerate(predictions):
            # Получаем название эмоции с максимальной вероятностью
            emotion = class_list[np.argmax(prediction)]

            # Вычисляем временной промежуток для текущего отрезка
            start_time = i * 2
            end_time = start_time + 2
            time_interval = f"{format_time(start_time)} - {format_time(end_time)}"

            # Добавляем в список результат в формате "название эмоции X-Y сек"
            prediction_with_times.append(f"{emotion} {time_interval}")

        return prediction_with_times

    # Если требуется одно итоговое предсказание
    final_prediction = np.sum(predictions, axis=0)
    return class_list[np.argmax(final_prediction)]


predict_emotion(prepare_prediction_batch('/content/audio_2024-11-02_11-08-56.ogg'), \
                emotional_testing_model, class_list=class_list_ru, return_list=True)

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:576: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 28 variables whereas the saved optimizer has 54 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 303ms/step


['отвращение 0 сек - 2 сек',
 'отвращение 2 сек - 4 сек',
 'отвращение 4 сек - 6 сек',
 'отвращение 6 сек - 8 сек',
 'отвращение 8 сек - 10 сек',
 'отвращение 10 сек - 12 сек',
 'отвращение 12 сек - 14 сек',
 'отвращение 14 сек - 16 сек',
 'отвращение 16 сек - 18 сек',
 'отвращение 18 сек - 20 сек',
 'отвращение 20 сек - 22 сек',
 'отвращение 22 сек - 24 сек',
 'отвращение 24 сек - 26 сек',
 'отвращение 26 сек - 28 сек',
 'отвращение 28 сек - 30 сек',
 'отвращение 30 сек - 32 сек',
 'отвращение 32 сек - 34 сек',
 'отвращение 34 сек - 36 сек',
 'отвращение 36 сек - 38 сек',
 'отвращение 38 сек - 40 сек',
 'отвращение 40 сек - 42 сек',
 'отвращение 42 сек - 44 сек',
 'отвращение 44 сек - 46 сек',
 'отвращение 46 сек - 48 сек',
 'отвращение 48 сек - 50 сек',
 'отвращение 50 сек - 52 сек',
 'отвращение 52 сек - 54 сек',
 'отвращение 54 сек - 56 сек',
 'отвращение 56 сек - 58 сек',
 'отвращение 58 сек - 1 мин 0 сек',
 'отвращение 1 мин 0 сек - 1 мин 2 сек',
 'отвращение 1 мин 2 сек - 1 мин 